In [1]:
# Import libraries
import pandas as pd
import numpy as np
import boto3

s3files_df = pd.read_csv('s3files.csv')
dbfiles_df = pd.read_csv('files.txt')
dbfiles_df.shape
# assign location to each dataframe that will we use in the merge
s3files_df['location'] = 's3'
dbfiles_df['location'] = 'db'
# append fully qualified path for s3 files
s3files_df['FileLocation'] = '915-profiler/temp/' + s3files_df['FileLocation'] 

In [2]:
print (s3files_df.shape)
print (dbfiles_df.shape)
s3files_df.head()
#dbfiles_df.head()

(131729, 2)
(57347, 2)


,FileLocation,location
0,915-profiler/temp/915ProfilerTempCNS.0505.2020...,s3
1,915-profiler/temp/915ProfilerTempCNS.0505.2020...,s3
2,915-profiler/temp/915ProfilerTempCNS.0505.2020...,s3
3,915-profiler/temp/915ProfilerTempCNS.0505.2020...,s3
4,915-profiler/temp/915ProfilerTempCNS.0505.2020...,s3


In [3]:
# We want a list of files that are on S3 but NOT in the DB, those files need to be deleted
# Let's do a left merge so that any files cleans will show up as NaN
files_merged = s3files_df.merge(dbfiles_df, on='FileLocation', how='left')


files_not_in_db_filter = files_merged['location_y'].isnull()
files_that_exist_on_s3_not_in_db = files_merged[files_not_in_db_filter]
print ('files_that_exist_on_s3_not_in_db', files_that_exist_on_s3_not_in_db.shape)
files_that_exist_on_s3_not_in_db.head()

files_that_exist_on_s3_not_in_db (74683, 3)


,FileLocation,location_x,location_y
2652,915-profiler/temp/915ProfilerTempCNS.0506.2020...,s3,NaN
2654,915-profiler/temp/915ProfilerTempCNS.0506.2020...,s3,NaN
2655,915-profiler/temp/915ProfilerTempCNS.0506.2020...,s3,NaN
2656,915-profiler/temp/915ProfilerTempCNS.0506.2020...,s3,NaN
29508,915-profiler/temp/915ProfilerTempCNS.0508.2020...,s3,NaN


In [16]:
files_that_exist_on_s3_not_in_db.iloc[2]['FileLocation']

'915-profiler/temp/915ProfilerTempCNS.0506.20200313043026.csv'

In [23]:
# Use the SDK to remove all files from S3 not in the DB
session = boto3.Session(
    region_name='us-gov-west-1',
    profile_name='vafb'
)

client = session.client('s3')
for objects in files_that_exist_on_s3_not_in_db.iterrows():
    key = objects[1]['FileLocation']
    response = client.delete_object(Bucket='dxhub-vafb-xui-weather-data-raw',Key=key)